In [3]:
%%time
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=24, inter_op_parallelism_threads=24)))

# import matplotlib.pyplot as plt
# import IPython.display as ipd

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
from time import time, ctime

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
from time import time, ctime

L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

#src folders
root_path = r'.'
out_path = r'.'
model_path = r'.'
train_data_path = os.path.join(root_path, 'data', 'train', 'audio')
test_data_path = os.path.join(root_path, 'data', 'test', 'audio')

Using TensorFlow backend.


CPU times: user 13.1 s, sys: 1.17 s, total: 14.3 s
Wall time: 29.7 s


In [6]:
# losses.binary_crossentropy?

In [7]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
#         elif label not in legal_labels:
#             nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT is simmetrical, so we take just the first half
    # FFT is also complex, to we take just the real part (abs)
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

### skip here

In [5]:
%%time
labels, fnames = list_wavs_fname(train_data_path)

new_sample_rate = 16000
y_train = []
x_train = []

n=0
for label, fname in zip(labels, fnames):
    n+=1
    if n%1000==0:
        print(int(n/1000), end='/{} '.format(int(len(labels)/1000)))
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    
    samples = pad_audio(samples)
    
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
        
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        
        y_train.append(label)
        x_train.append(specgram)
        
x_train = np.array(x_train)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(y_train)

label_index = y_train.columns.values

y_train = y_train.values
y_train = np.array(y_train)
del labels, fnames
gc.collect()

X = x_train.copy()
Y = y_train.copy()
print (X.shape, Y.shape, type(X), type(Y))

./data/train/audio
1/64 2/64 3/64 4/64 5/64 6/64 7/64 8/64 9/64 10/64 11/64 

/home/shj16110/anaconda2/envs/3.6.1/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


12/64 13/64 14/64 15/64 16/64 17/64 18/64 19/64 20/64 21/64 22/64 23/64 24/64 25/64 26/64 27/64 28/64 29/64 30/64 31/64 32/64 33/64 34/64 35/64 36/64 37/64 38/64 39/64 40/64 41/64 42/64 43/64 44/64 45/64 46/64 47/64 48/64 49/64 50/64 51/64 52/64 53/64 54/64 55/64 56/64 57/64 58/64 59/64 60/64 61/64 62/64 63/64 64/64 (64841, 99, 161, 1) (64841, 31) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
CPU times: user 1min 49s, sys: 7.31 s, total: 1min 56s
Wall time: 17min 19s


In [13]:
label_index = np.array(list(['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go',
       'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on',
       'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop',
       'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'])).astype('object')

In [16]:
# %%time
# np.save('X_31label_16k.npy', X)
# np.save('Y_31label_16k.npy', Y)

CPU times: user 0 ns, sys: 877 ms, total: 877 ms
Wall time: 1.53 s


### mid start from here

In [8]:
%%time
X = np.load('X_31label_16k.npy')
Y = np.load('Y_31label_16k.npy')

CPU times: user 0 ns, sys: 1.39 s, total: 1.39 s
Wall time: 2.52 s


In [15]:
input_shape = (99, 161, 1)
nclass = 31
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(16, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(16, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(21, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(21, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(36, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(3, 3))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = BatchNormalization()(Dense(256, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 99, 161, 1)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 99, 161, 1)        4         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 98, 160, 16)       80        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 97, 159, 16)       1040      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 79, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 79, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 46, 77, 21)        3045      
__________

https://machinelearningmastery.com/check-point-deep-learning-models-keras/

In [16]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
# checkpoint
filepath="models/F36W-31L-{epoch:02d}-{val_acc:.5f}_.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early_stopping]

In [17]:
S = lambda *x: [print(i.shape, type(i)) for i in x]
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.1, random_state=2018)
S(x_train, y_train, x_valid, y_valid)

(58356, 99, 161, 1) <class 'numpy.ndarray'>
(58356, 31) <class 'numpy.ndarray'>
(6485, 99, 161, 1) <class 'numpy.ndarray'>
(6485, 31) <class 'numpy.ndarray'>


[None, None, None, None]

In [18]:
pd.DataFrame(y_valid).sum().describe()

count     31.000000
mean     209.193548
std       47.781739
min       16.000000
25%      183.500000
50%      225.000000
75%      240.500000
max      261.000000
dtype: float64

In [20]:
%%time
# Fit the model
model.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), 
          epochs=100, shuffle=True, verbose=1, callbacks=callbacks_list)

Train on 58356 samples, validate on 6485 samples
Epoch 1/100
58356/58356 [==============================] - 131s - loss: 0.2515 - acc: 0.9226 - val_loss: 0.2531 - val_acc: 0.9263
Epoch 2/100
58356/58356 [==============================] - 133s - loss: 0.2472 - acc: 0.9245 - val_loss: 0.2471 - val_acc: 0.9269
Epoch 3/100
58356/58356 [==============================] - 131s - loss: 0.2425 - acc: 0.9255 - val_loss: 0.2290 - val_acc: 0.9343
Epoch 4/100
58356/58356 [==============================] - 133s - loss: 0.2440 - acc: 0.9252 - val_loss: 0.2334 - val_acc: 0.9338
Epoch 5/100
58356/58356 [==============================] - 131s - loss: 0.2365 - acc: 0.9280 - val_loss: 0.2344 - val_acc: 0.9308
Epoch 6/100
58356/58356 [==============================] - 132s - loss: 0.2303 - acc: 0.9292 - val_loss: 0.2315 - val_acc: 0.9363
Epoch 7/100
58356/58356 [==============================] - 131s - loss: 0.2281 - acc: 0.9304 - val_loss: 0.2274 - val_acc: 0.9346
Epoch 8/100
58356/58356 [================

In [23]:
# pd.DataFrame(y_valid).sum()

In [29]:
# model.save(os.path.join(model_path, 'cnn.model'))

In [21]:
!ls models|grep 36W

F36W-31L-00-0.79630_.hdf5
F36W-31L-00-0.98669_.hdf5
F36W-31L-01-0.86507_.hdf5
F36W-31L-01-0.99168_.hdf5
F36W-31L-02-0.89067_.hdf5
F36W-31L-02-0.93431_.hdf5
F36W-31L-02-0.99251_.hdf5
F36W-31L-03-0.89345_.hdf5
F36W-31L-04-0.89653_.hdf5
F36W-31L-05-0.90763_.hdf5
F36W-31L-05-0.93631_.hdf5
F36W-31L-06-0.91781_.hdf5
F36W-31L-08-0.92290_.hdf5
F36W-31L-12-0.93739_.hdf5
F36W-31L-13-0.92907_.hdf5
F36W-31L-13-0.93863_.hdf5
F36W-31L-14-0.93169_.hdf5
F36W-31L-16-0.93416_.hdf5
F36W-31L-17-0.94048_.hdf5


### predict

In [22]:
Mname = 'models/F36W-31L-17-0.94048_.hdf5'

# returns a compiled model
# identical to the previous one
model = load_model(Mname)

legal_label = np.array(['down', 'go', 'left', 'no', 'off', 'on', 'right',
                        'silence', 'stop', 'unknown', 'up', 'yes']).astype('object')

In [25]:
def test_data_generator(batch=16):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('\\')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
        yield fnames, imgs
    raise StopIteration()
    
new_sample_rate=16000

label_index = np.array(list(['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go',
       'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on',
       'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop',
       'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'])).astype('object')

In [27]:
%%time

start = time()
batch = 64

#exit() #delete this
#del x_train, y_train
gc.collect()

index = []
results = []
N=0
for fnames, imgs in test_data_generator(batch=batch):
    N+=1
    if N%10==0:
        print ('used: {:.2f} s'.format(time()-start), end=' :{} '.format(N*batch))
        start = time()
    predicts = model.predict(imgs)
    predicts = np.argmax(predicts, axis=1)
    predicts = [label_index[p] for p in predicts]
    index.extend(fnames)
    results.extend(predicts)

df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results


df['fname'] = df['fname'].apply(lambda x:x.split('audio/')[-1])
# df.to_csv(os.path.join(out_path, Mname.split('/')[-1]+'_submission.csv'), index=False)

legal_label = np.array(['down', 'go', 'left', 'no', 'off', 'on', 'right',
                        'silence', 'stop', 'up', 'yes']).astype('object')
df['label'].apply(lambda x: 'unknown' if x not in legal_label else x)
df.to_csv(os.path.join(out_path, Mname.split('/')[-1]+'_submission.csv'), index=False)

used: 13.38 s :640 used: 12.36 s :1280 used: 12.49 s :1920 used: 12.64 s :2560 used: 13.15 s :3200 used: 12.99 s :3840 used: 12.73 s :4480 used: 13.19 s :5120 used: 12.61 s :5760 used: 12.95 s :6400 used: 12.38 s :7040 used: 13.26 s :7680 used: 13.35 s :8320 used: 12.91 s :8960 used: 22.01 s :9600 used: 25.01 s :10240 used: 15.79 s :10880 used: 16.66 s :11520 used: 17.45 s :12160 used: 15.16 s :12800 used: 15.36 s :13440 used: 14.92 s :14080 used: 13.48 s :14720 used: 13.47 s :15360 used: 13.15 s :16000 used: 12.76 s :16640 used: 13.35 s :17280 used: 13.06 s :17920 used: 13.25 s :18560 used: 13.00 s :19200 used: 13.22 s :19840 used: 13.06 s :20480 used: 12.62 s :21120 used: 12.87 s :21760 used: 13.00 s :22400 used: 13.07 s :23040 used: 13.58 s :23680 used: 13.09 s :24320 used: 12.77 s :24960 used: 13.27 s :25600 used: 13.05 s :26240 used: 12.74 s :26880 used: 12.78 s :27520 used: 12.98 s :28160 used: 12.58 s :28800 used: 12.34 s :29440 used: 12.67 s :30080 used: 14.11 s :30720 used: 13

/home/shj16110/anaconda2/envs/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: generator 'test_data_generator' raised StopIteration
  if sys.path[0] == '':


CPU times: user 5min 6s, sys: 20.4 s, total: 5min 26s
Wall time: 59min 53s


In [30]:
legal_label = np.array(['down', 'go', 'left', 'no', 'off', 'on', 'right',
                        'silence', 'stop', 'up', 'yes']).astype('object')
df['label'] = df['label'].apply(lambda x: 'unknow' if x not in legal_label else x)
df.to_csv(os.path.join(out_path, Mname.split('/')[-1]+'_submission.csv'), index=False)

In [31]:
df['label'].value_counts()

unknow     81834
right      14209
off         8725
no          7283
on          7265
up          5950
silence     5779
yes         5698
left        5632
go          5581
stop        5350
down        5232
Name: label, dtype: int64

In [32]:
Mname

'models/F36W-31L-17-0.94048_.hdf5'